In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from matplotlib.pyplot import plot


In [ ]:
#load dataset
fn2 = "dataset_sst2_eval.txt"
fn = "dataset_sst2_train.txt"

nparr2 = np.loadtxt(fn2, delimiter=",")
nparr = np.loadtxt(fn, delimiter=",")

In [ ]:
#split 80/20
#whether to average entropies up to start_layer or just use entropy at start layer
avg = 0 
#layer after which you make entropy prediction
start_layer = 6 #indexed 0-11

test = nparr2[:,:11]
train = nparr[:,:11]

if avg:
  (x_train, y_train) = (train[:,:start_layer], train[:,start_layer+1:])
  (x_test, y_test) = (test[:,:start_layer], test[:,start_layer+1:])
  x_train = np.mean(x_train, axis=-1)
  x_test = np.mean(x_test, axis=-1)
else:
  (x_train, y_train) = (train[:,start_layer], train[:,start_layer+1:])
  (x_test, y_test) = (test[:,start_layer], test[:,start_layer+1:])

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = 1)) #input layer
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dense(64, activation='linear'))
model.add(Dense(10-start_layer)) # output layer

model.compile(
    optimizer='adam',
    loss='mean_squared_error' 
)

model.fit(x_train, y_train, epochs=3, verbose=1)

Epoch 1/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0497
Epoch 2/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0484
Epoch 3/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0483


In [ ]:
#check prediction
val_loss = model.evaluate(x_test, y_test)
#model.summary()

  

307/307 [==============================] - 0s 924us/step - loss: 0.0505


In [ ]:
#produce lookup table
table = np.zeros((100, 11-start_layer))
for i in range(0,100):
  xval = i*0.01;
  yval = model.predict(np.array([xval]), batch_size=1)
  table[i,0] = xval
  table[i,1:] = yval[0]

threshold_indices = table < 0
table[threshold_indices] = 0

np.savetxt("sst2_lookup_table_opt.csv", table, delimiter=",")